## ユーザー情報の取得

In [16]:
import json
import time
import random

import numpy as np
import pandas as pd
from requests_oauthlib import OAuth1Session

In [20]:
# APIの取得
def getAPI():
    # キー取得
    with open('key.json') as f:
        keys = json.load(f)
    # 環境変数から承認情報を取得
    CONSUMER_KEY    = keys['CONSUMER_KEY']
    CONSUMER_SECRET = keys['CONSUMER_SECRET']
    ACCESS_TOKEN    = keys['ACCESS_TOKEN']
    ACCESS_SECRET   = keys['ACCESS_SECRET']
    api = OAuth1Session(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_SECRET)
    return api

In [35]:
# 特定ユーザーフォロワーIdを取得
def getFollowerIds(api, screen_name):
    ids = []
    cursor = -1
    
    url = 'https://api.twitter.com/1.1/followers/ids.json'
    params = {'screen_name': screen_name}
    req = api.get(url, params=params)
    
    if req.status_code == 200:
        while cursor != 0:
            params['cursor'] = str(cursor)
            temp = json.loads(req.text)

            ids.extend(temp['ids'])
            cursor = temp['next_cursor']
            time.sleep(10*random.uniform(0.5,1.5))
    else:
        print ("Error: %d at getFollowerIds" % req.status_code)
    return ids

In [36]:
api = getAPI()

ids = getFollowerIds(api, screen_name='henjinmajime')
len(ids)

KeyboardInterrupt: 

In [37]:
# idsをnpyファイルで出力
np.save('follower_ids.npy', ids)

### ユーザー情報をcsv出力する。
- 取得情報  
user_id/screen_name/num_friend/num_follower/self_description

In [99]:
# idから上記の情報を取得
def getUserInfo(api, ids):
    url = 'https://api.twitter.com/1.1/users/lookup.json'
    ids = np.array(ids).astype(str)
    user_info = []
    
    for i in range(0, len(ids), 100):
        _ids = ','.join(ids[i:i+100])
        
        params = {'user_id': _ids}
        
        req = api.get(url, params=params)
        if req.status_code == 200:
            req_text = json.loads(req.text)
            user_info.extend([[user['id'], user['screen_name'], user['friends_count'], user['followers_count'], user['description']] for user in req_text])
        else:
            print ("Error: %d at getUserInfo()" % req.status_code)
        time.sleep(10*random.uniform(0.5,1.5))
    return user_info

In [100]:
user_info = getUserInfo(api, ids)

In [101]:
df = pd.DataFrame(user_info, columns=['id', 'screen_name', 'friends_count', 'followers_count', 'self_description'])

In [112]:
df.to_csv('./data/user_info.csv', index=False)

In [115]:
df = pd.read_csv('./data/user_info.csv')

In [116]:
df

,id,screen_name,friends_count,followers_count,self_description
0,987949359751872512,EIUmFkEw2avNp5q,367,86,天野碧海です。育英のf3です。よろしく。
1,705310264334233601,mihooneoklove,440,746,トプ右｜17｜西浜高校⇒通信(LJK)｜ピアス｜ONE OK ROCK｜天竜＆コンビニバイター
2,1389240854,s0j1_mkt,707,558,アクティブに٩( *˙0˙*)۶時間は有限⏳フットワークはとっても軽く、やりたいことはぜんぶ...
3,827738111132131329,kotaro200216,168,223,高1 I.E バド部🏸 もっと強くなりたい、 @mip0000o推し インスタ率高め
4,875528323991969792,lli___eguyu25,130,60,い り の み ゆ ♡ え ぐ ち た く や\n- - - - -Kiramune- ...
5,989171553622409216,__michan46__,72,8,ただの乃木オタのリプ垢 ?¿
6,990496542455775233,q41nW2ZNGRJVulZ,21,7,ミニオンLOVE shs130
7,975660997796028416,h9U4rFhcfC3Katz,82,28,NaN
8,812874132656308224,mi_sa_7913,440,381,3-C 10キロ無くなれ。
9,990242598731202560,yumena69465329,2,0,ヘンジンマジメ/ぎんしゃむくん大好き #イラスト #ぎんしゃむ #ヘンジンマジメ\n絵描き...
